In [122]:
options(stringsAsFactors=F)
options(max.print=1000)
suppressPackageStartupMessages(library(data.table))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(ggsci))
suppressPackageStartupMessages(library(ggrepel))
suppressPackageStartupMessages(library(gridExtra))
suppressPackageStartupMessages(library(RColorBrewer))
suppressPackageStartupMessages(library(circlize))
suppressPackageStartupMessages(library(ggpubr))
suppressPackageStartupMessages(library(corrplot))
suppressPackageStartupMessages(library(igraph))
suppressPackageStartupMessages(library(cowplot))
suppressPackageStartupMessages(library(ggsignif))
suppressPackageStartupMessages(library(ggpubr))
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(scales))
suppressPackageStartupMessages(library(ComplexHeatmap))
suppressPackageStartupMessages(library(VennDiagram))
suppressPackageStartupMessages(library(qvalue))
options(bitmapType = 'cairo', device = 'png')

setwd('/psycl/g/mpsziller/lucia/CAD_UKBB/eQTL_PROJECT')
fold_notebook <- '/psycl/g/mpsziller/lucia/castom_cad_scz/jupyter_notebook/'
fold_cl <- "OUTPUT_GTEx/predict_CAD/Liver/200kb/CAD_GWAS_bin5e-2/UKBB/devgeno0.01_testdevgeno0/CAD_HARD_clustering/update_corrPCs/"
file_cl <- "OUTPUT_GTEx/predict_CAD/Liver/200kb/CAD_GWAS_bin5e-2/UKBB/devgeno0.01_testdevgeno0/CAD_HARD_clustering/update_corrPCs/tscore_corrPCs_zscaled_clusterCases_PGmethod_HKmetric.RData"
fold <- "OUTPUT_GTEx/predict_CAD/AllTissues/200kb/CAD_GWAS_bin5e-2/UKBB/"
filt_path_file <- "pathOriginal_filtJS0.2_corrPCs_tscoreClusterCases_featAssociation.RData"
outFold <- sprintf('%sCAD_clustering/', fold_notebook)

cl <- get(load(file_cl))
n_gr <- length(unique(cl$cl_best$gr))
tscore_gr <- list()
pathR_gr <- list()
pathGO_gr <- list()
path_gr <- list()

for(i in 1:n_gr){
    tscore_gr[[i]] <- read.table(sprintf('%scluster_specific_PALAS/tscore_pval_ClusterCasesVSControls_gr%i.txt', fold_cl, i), 
                                 header=T, sep = "\t", stringsAsFactors = F) 
    tscore_gr[[i]] <- tscore_gr[[i]] %>% 
        dplyr::mutate(feat_tissue = paste0(ensembl_gene_id, "_", tissue))

    pathR_gr[[i]] <- read.delim(sprintf('%scluster_specific_PALAS/path_Reactome_pval_ClusterCasesVSControls_gr%i_filt.txt', fold_cl, i), 
                                header=T, sep = "\t", stringsAsFactors = F)
    pathGO_gr[[i]] <- read.delim(sprintf('%scluster_specific_PALAS/path_GO_pval_ClusterCasesVSControls_gr%i_filt.txt', fold_cl, i), 
                                header=T, sep = "\t", stringsAsFactors = F)

    colnames_path <- intersect(colnames(pathR_gr[[i]]), colnames(pathGO_gr[[i]]))

    pathR_gr[[i]] <- pathR_gr[[i]] %>% 
        dplyr::select(all_of(colnames_path)) %>%
        dplyr::mutate(feat_tissue = paste0(path, "_", tissue)) %>%
        dplyr::mutate(type = "Reactome")

    pathGO_gr[[i]] <- pathGO_gr[[i]] %>% 
        dplyr::select(all_of(colnames_path)) %>%
        dplyr::mutate(feat_tissue = paste0(path, "_", tissue)) %>%
        dplyr::mutate(type = "GO")
    
    path_gr[[i]] <- bind_rows(
        pathR_gr[[i]], pathGO_gr[[i]]) %>%
        dplyr::mutate(full_id = paste(feat_tissue, type, sep = "_"))
    
}

endop_tissue <- read_tsv(sprintf('%s/filter_endopheno/tscore_corrPCs_zscaled_clusterCases_PGmethod_HKmetric_phenoAssociation_GLM_combined_keepPhenoClass.txt', fold_cl), 
                        show_col_types = FALSE)
pheno_class <- unique(endop_tissue$pheno_type)
# get only significant endophenotypes classes
pheno_class_sign <- endop_tissue %>% filter(pval_corr <= 0.1) %>% pull(pheno_type) %>% unique() 

if("ICD9-10_OPCS4" %in% pheno_class){
    pheno_class <- c(pheno_class, "ICD10_Circulatory_system", "ICD10_Endocrine")
    pheno_class <- setdiff(pheno_class, "ICD9-10_OPCS4")
}
pheno_class <- setdiff(pheno_class, c("Height_derived", "Early_life_factors")) # no PALAS computed for this endophenotype
pheno_class 

tissues <- unique(tscore_gr[[1]]$tissue)

[1] "Blood_biochemistry"       "Blood_count"             
 [3] "Blood_pressure"           "Body_size_measures"      
 [5] "Impedance_measures"       "Arterial_stiffness"      
 [7] "Hand_grip_strength"       "Blood_count_ratio"       
 [9] "Family_history"           "ICD10_Circulatory_system"
[11] "ICD10_Endocrine"

In [123]:
# create a function to remove pathway with 1 gene and recompute pvalues
recompte_path <- function(tissues_name, res, id_pval){
  tmp <- lapply(tissues_name, function(x) res[res$tissue == x & res$ngenes_tscore>1,])
  for(i in 1:length(tmp)){
    tmp[[i]][, id_pval+1] <- qvalue(tmp[[i]][, id_pval])$qvalue
    tmp[[i]][, id_pval+2] <- p.adjust(tmp[[i]][, id_pval], method = 'BH')
  }
  tmp <- do.call(rbind, tmp)
  return(tmp)
}

# create function to load all TWAS and PALAS results per phenotype class
load_TWAS_PALAS_perclass <- function(tissues, pheno_id){

  df_tscore <- df_pathR <- df_pathGO <- list()
  for(i in 1:length(tissues)){
  
    #print(t)

    t <- tissues[i]
    fold_PALAS <- sprintf("OUTPUT_GTEx/predict_CAD/%s/200kb/CAD_GWAS_bin5e-2/UKBB/devgeno0.01_testdevgeno0/", t)
    
    if(file.exists(sprintf("%spval_%s_withMed_pheno_covCorr.RData", fold_PALAS, pheno_id))){
      res_file <- sprintf("%spval_%s_withMed_pheno_covCorr.RData", fold_PALAS, pheno_id)
    }else{
      res_file <- sprintf("%spval_%s_pheno_covCorr.RData", fold_PALAS, pheno_id)
    }
    
    tmp <- get(load(res_file))
    n_pheno <- nrow(tmp$pheno)

    df_tscore[[i]] <- list()
    df_pathR[[i]] <- list()
    df_pathGO[[i]] <- list()

    for(j in 1:n_pheno){

      tmp$tscore[[j]]$tissue <- t
      tmp$pathScore_reactome[[j]]$tissue <- t
      tmp$pathScore_GO[[j]]$tissue <- t
      tmp$pathScore_reactome[[j]]$genes_path <- tmp$pathScore_reactome[[j]]$improvement_sign <- NA
      tmp$pathScore_GO[[j]]$genes_path <- tmp$pathScore_GO[[j]]$improvement_sign <- NA

    for(k in 1:nrow(tmp$pathScore_reactome[[j]])){
      tmp$pathScore_reactome[[j]]$genes_path[k] <- paste0(tmp$info_pathScore_reactome[[j]][[k]]$tscore$external_gene_name, collapse = ',')
      tmp$pathScore_reactome[[j]]$improvement_sign[k] <- all(tmp$info_pathScore_reactome[[j]][[k]]$tscore[,8] > tmp$pathScore_reactome[[j]][k,13])
    }

    for(k in 1:nrow(tmp$pathScore_GO[[j]])){
      tmp$pathScore_GO[[j]]$genes_path[k] <- paste0(tmp$info_pathScore_GO[[j]][[k]]$tscore$external_gene_name, collapse = ',')
      tmp$pathScore_GO[[j]]$improvement_sign[k] <- all(tmp$info_pathScore_GO[[j]][[k]]$tscore[,8] > tmp$pathScore_GO[[j]][k,15])
    }
    df_tscore[[i]][[j]] <- tmp$tscore[[j]]
    df_pathR[[i]][[j]] <- tmp$pathScore_reactome[[j]]
    df_pathGO[[i]][[j]] <- tmp$pathScore_GO[[j]]
    }
  }
  df_tscore_all <- lapply(1:n_pheno, function(x) do.call(rbind, lapply(1:length(tissues), function(y) df_tscore[[y]][[x]])))
  df_pathR_all <- lapply(1:n_pheno, function(x) do.call(rbind, lapply(1:length(tissues), function(y) df_pathR[[y]][[x]])))
  df_pathGO_all <- lapply(1:n_pheno, function(x) do.call(rbind, lapply(1:length(tissues), function(y) df_pathGO[[y]][[x]])))

  # filter out pathways with only 1 gene
  df_pathR_all_red <- lapply(df_pathR_all, function(x) recompte_path(res = x, tissues_name = tissues, id_pval = 13))
  df_pathGO_all_red <- lapply(df_pathGO_all, function(x) recompte_path(res = x, tissues_name = tissues, id_pval = 15))

  out <- list(pheno = tmp$pheno %>% mutate(pheno_type = pheno_id), 
              tscore = df_tscore_all, 
              pathR = df_pathR_all_red, 
              pathGO = df_pathGO_all_red)
  return(out)
}

In [124]:
# load TWAS and PALAS for endophenotypes
out <- list()
for(i in 1:length(pheno_class)){
  pheno_id <- pheno_class[i]
  print(pheno_id)
  out[[i]] <- load_TWAS_PALAS_phenoclass(tissues, pheno_id)
}

In [70]:
# create a list of dataframes, each dataframe contains all the pathways/tscore results across all tissues for a single phenotype
# the order of phenotypes in list is the same as they appear in pheno_all

pheno_all <- do.call(rbind, lapply(1:length(out), function(x) out[[x]]$pheno))

tscore_all <- unlist(lapply(1:length(out), function(x) out[[x]]$tscore), recursive = FALSE)
pathR_all <-  unlist(lapply(1:length(out), function(x) out[[x]]$pathR), recursive = FALSE)
pathGO_all <-  unlist(lapply(1:length(out), function(x) out[[x]]$pathGO), recursive = FALSE)

path_all <- list()
for(i in 1:nrow(pheno_all)){
    
    colnames_path <- intersect(colnames(pathR_all[[i]]), colnames(pathGO_all[[i]]))

    pathR_all[[i]] <- pathR_all[[i]] %>% 
        dplyr::select(all_of(colnames_path)) %>%
        dplyr::mutate(feat_tissue = paste0(path, "_", tissue)) %>%
        dplyr::mutate(type = "Reactome")

    pathGO_all[[i]] <- pathGO_all[[i]] %>% 
        dplyr::select(all_of(colnames_path)) %>%
        dplyr::mutate(feat_tissue = paste0(path, "_", tissue)) %>%
        dplyr::mutate(type = "GO")
    
    path_all[[i]] <- bind_rows(
        pathR_all[[i]], pathGO_all[[i]]) %>%
        dplyr::mutate(full_id = paste(feat_tissue, type, sep = "_"))
}


In [109]:
library(epitools)
pval_corr_thr <- 0.05
id_zstat <- 12
id_pval_corr <- 15

# compute correlations for each pair of group and endophenotype
corr_res <- list()
OR_res <- list()

for(i in 1:n_gr){
    # compute person correlation of Zstat
    corr_res[[i]] <- pheno_all %>% 
        mutate(group = paste0("gr", i)) %>%
        mutate(corr = sapply(path_all, function(x) cor.test(path_gr[[i]][, id_zstat], x[, id_zstat])$estimate)) %>%
        mutate(corr_pvalue = sapply(path_all, function(x) cor.test(path_gr[[i]][, id_zstat], x[, id_zstat])$p.value)) 
    
    # compute enrichment of significant results, put NA if no significant results group or phenotype (due to small sample size)
    # not informative on the sign!!!
    df <- list()
    for(j in 1:nrow(pheno_all)){
        data <- table(path_all[[j]][, id_pval_corr] <= pval_corr_thr, path_gr[[i]][,id_pval_corr] <= pval_corr_thr)
        dimnames(data) <- list('Group' = c("FDR > 0.05", "FDR < 0.05"), 'Pheno' = c("FDR > 0.05", "FDR < 0.05"))
        tryCatch(tmp <- as.data.frame(t(oddsratio(data)$measure["FDR < 0.05",])) %>% 
            mutate(pvalue = oddsratio(data)$p.value["FDR < 0.05", 2], gr = paste0("gr", i)), 
            error = function(e) tmp <- data.frame(estimate = NA, lower = NA, upper = NA, pvalue = NA, gr = paste0("gr", i)))
        df[[j]] <- cbind(pheno_all[i,, drop = F], tmp) 

    }
    df <- bind_rows(df)
    OR_res[[i]] <- df
   
}
corr_res <- bind_rows(corr_res) 
OR_res <- bind_rows(OR_res)


Warning message in chisq.test(xx, correct = correction):
“Chi-squared approximation may be incorrect”
Warning message in chisq.test(xx, correct = correction):
“Chi-squared approximation may be incorrect”
Warning message in chisq.test(xx, correct = correction):
“Chi-squared approximation may be incorrect”
Warning message in chisq.test(xx, correct = correction):
“Chi-squared approximation may be incorrect”
Warning message in chisq.test(xx, correct = correction):
“Chi-squared approximation may be incorrect”
Warning message in chisq.test(xx, correct = correction):
“Chi-squared approximation may be incorrect”
Warning message in chisq.test(xx, correct = correction):
“Chi-squared approximation may be incorrect”
Warning message in chisq.test(xx, correct = correction):
“Chi-squared approximation may be incorrect”
Warning message in chisq.test(xx, correct = correction):
“Chi-squared approximation may be incorrect”
Warning message in chisq.test(xx, correct = correction):
“Chi-squared approximatio

In [112]:
OR_res[order(OR_res$estimate, decreasing = T),]

,pheno_id,FieldID,Field,Path,Strata,Sexed,Coding,Coding_meaning,original_type,transformed_type,nsamples,nsamples_T,nsamples_F,estimate,lower,upper,pvalue,gr
,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
204,30630,30630,Apolipoprotein A,Biological samples > Assay results > Blood assays > Blood biochemistry,Primary,Unisex,NA,NA,CONTINUOUS_MAIN,CONTINUOUS,296255,NA,NA,182.120342,37.978807,4332.863292,7.198766e-20,gr4
82,30610,30610,Alkaline phosphatase,Biological samples > Assay results > Blood assays > Blood biochemistry,Primary,Unisex,NA,NA,CONTINUOUS_MAIN,CONTINUOUS,325373,NA,NA,172.101055,35.890538,4095.569875,1.907843e-19,gr2
143,30620,30620,Alanine aminotransferase,Biological samples > Assay results > Blood assays > Blood biochemistry,Primary,Unisex,NA,NA,CONTINUOUS_MAIN,CONTINUOUS,325251,NA,NA,32.563196,11.294871,82.570554,1.807647e-07,gr3
249,30640,30640,Apolipoprotein B,Biological samples > Assay results > Blood assays > Blood biochemistry,Primary,Unisex,NA,NA,CONTINUOUS_MAIN,CONTINUOUS,323788,NA,NA,28.837393,21.386428,39.540375,6.147133e-129,gr5
263,30640,30640,Apolipoprotein B,Biological samples > Assay results > Blood assays > Blood biochemistry,Primary,Unisex,NA,NA,CONTINUOUS_MAIN,CONTINUOUS,323788,NA,NA,18.671739,14.287277,24.603447,3.226208e-101,gr5
5,30600,30600,Albumin,Biological samples > Assay results > Blood assays > Blood biochemistry,Primary,Unisex,NA,NA,CONTINUOUS_MAIN,CONTINUOUS,298031,NA,NA,14.224161,11.294473,18.001924,1.399923e-108,gr1
129,30620,30620,Alanine aminotransferase,Biological samples > Assay results > Blood assays > Blood biochemistry,Primary,Unisex,NA,NA,CONTINUOUS_MAIN,CONTINUOUS,325251,NA,NA,13.210983,10.783982,16.138463,9.989023e-100,gr3
19,30600,30600,Albumin,Biological samples > Assay results > Blood assays > Blood biochemistry,Primary,Unisex,NA,NA,CONTINUOUS_MAIN,CONTINUOUS,298031,NA,NA,12.044268,9.612272,15.123329,4.419928e-93,gr1
254,30640,30640,Apolipoprotein B,Biological samples > Assay results > Blood assays > Blood biochemistry,Primary,Unisex,NA,NA,CONTINUOUS_MAIN,CONTINUOUS,323788,NA,NA,11.046482,8.518124,14.403687,1.235734e-70,gr5


In [107]:
j=1
data <- table(path_all[[j]][, id_pval_corr] <= pval_corr_thr, path_gr[[i]][,id_pval_corr] <= pval_corr_thr)
        dimnames(data) <- list('Group' = c("FDR > 0.05", "FDR < 0.05"), 'Pheno' = c("FDR > 0.05", "FDR < 0.05"))

t(oddsratio(data)$measure["FDR < 0.05",])

estimate,lower,upper
1.676017,1.295043,2.147774
